In [ ]:
# !pip install --upgrade openai
# !pip install openai==0.28

In [ ]:
!python run_api_gen_data.py --prompt_name gen_scene --character CassiusDio

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="gemma",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    # {"role": "user", "content": "Who won the world series in 2020?"},
    # {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    # {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

In [ ]:
# Ollama
# client = OpenAI(
# base_url = 'http://localhost:11434/v1',
# api_key='ollama', # required, but unused
# )
# response = client.chat.completions.create(
#     model="gemma",
#     messages=[
#         {"role": "system", "content": sys_prompt_content},
#         {"role": "user", "content": input},
#         ],
#     max_tokens=max_tokens,
#     temperature=temperature,
#     top_p=top_p,
#     frequency_penalty=frequency_penalty,
#     presence_penalty=presence_penalty,
#     n=n,
#     stop=stop,
# )

In [ ]:
from groq import Groq

client = Groq(
    api_key='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',
)

completion = client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[
        {
            "role": "user",
            "content": ""
        }
    ],
    temperature=0.5,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


In [ ]:
# !python parser/parse_data_scene.py result/2023-10-08/gen_scene/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
# !python run_api_gen_data.py --prompt_name gen_dialogue --character CassiusDio --data_path processed/2023-10-08/

In [ ]:
# !python parser/parse_data_dialogue.py result/2023-10-08/gen_dialogue/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
# python run_api_gen_data.py --prompt_name gen_hallucination --character CassiusDio --data_path processed/2023-10-08/

In [ ]:
# python parser/parse_data_hallucination.py result/2023-10-08/gen_hallucination/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
python parser/convert_prompt_data.py processed/2023-10-08/generated_agent_dialogue_CassiusDio.json

In [ ]:
# conda install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
# conda install cuda -c nvidia
# pip3 install "fschat[model_worker,webui]"
# git clone https://github.com/lm-sys/FastChat.git
# cd FastChat
# pip3 install -e ".[train]"

# cd /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM
# conda activate vllmenv



# ENVARS ADDED **ONLY FOR READABILITY**
NVIDIA_CUDA_PPA=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/
NVIDIA_CUDA_PREFERENCES=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
NVIDIA_CUDA_PUBKEY=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/7fa2af80.pub

# Add NVIDIA Developers 3rd-Party PPA
sudo wget ${NVIDIA_CUDA_PREFERENCES} -O /etc/apt/preferences.d/nvidia-cuda
sudo apt-key adv --fetch-keys ${NVIDIA_CUDA_PUBKEY}
echo "deb ${NVIDIA_CUDA_PPA} /" | sudo tee /etc/apt/sources.list.d/nvidia-cuda.list

# Install development tools
sudo apt update
sudo apt install -y cuda

In [ ]:
import json

train_json = json.load(open('processed\\2023-10-08\\prompted\\prompted_agent_dialogue_CassiusDio.jsonl', "r"))

In [ ]:
# cd C:\Users\rs659\Desktop\Pliny\Character-LLM


# Scene Extraction
# python run_api_gen_data.py --prompt_name gen_scene --character Beethoven --debug 1
# python parser/parse_data_scene.py result/2023-10-08/gen_scene/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl

# Experience Completion
# python run_api_gen_data.py --prompt_name gen_dialogue --character Beethoven --data_path processed/2023-10-08/ --debug 1
# python parser/parse_data_dialogue.py result/2023-10-08/gen_dialogue/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl

# Protective Scene
# python run_api_gen_data.py --prompt_name gen_hallucination --character Beethoven --data_path processed/2023-10-08/ --debug 1
# python parser/parse_data_hallucination.py result/2023-10-08/gen_hallucination/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl


# Convert to Training Format
# python parser/convert_prompt_data.py processed/2023-10-08/generated_agent_dialogue_Beethoven.json


In [ ]:
# conda create -n finetuneenv python=3.9
# pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121



python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path '/mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl' \
    --output_dir './lora-alpaca'

## Dataset format Fastchat

In [ ]:
class ProcessedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, raw_data, tokenizer: transformers.PreTrainedTokenizer):
        super(ProcessedDataset, self).__init__()

        rank0_print("Formatting inputs...")
        sources = [ex['prompt'] for ex in raw_data]
        targets = [ex['output'] for ex in raw_data]
        data_dict = alpaca_preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i],
        )



import numpy as np

local_rank = None
def rank0_print(*args):
    if local_rank == 0:
        print(*args)


# data='/mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl'
data='C:/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl'
raw_data =[]
with open(data, 'r', encoding='utf-8') as fp:
    for line in fp:
        if line:
            raw_data.append(json.loads(line))
perm = np.random.permutation(len(raw_data))
split = int(len(perm) * 0.98)                                                                                                               
train_indices = perm[:split]
eval_indices = perm[split:]
train_raw_data = [raw_data[i] for i in train_indices]
eval_raw_data = [raw_data[i] for i in eval_indices]
rank0_print(f"#train {len(train_raw_data)}, #eval {len(eval_raw_data)}")

# train_dataset = dataset_cls(train_raw_data, tokenizer=tokenizer)
# eval_dataset = dataset_cls(eval_raw_data, tokenizer=tokenizer)
# return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

## Custom Dataset and training

In [ ]:
# https://medium.com/@iamarunbrahma/fine-tuning-of-falcon-7b-large-language-model-using-qlora-on-mental-health-dataset-aa290eb6ec85

# Windows Environent
# conda create -n plinyenv python=3.9
# conda activate plinyenv
# pip install transformers
# pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
# pip install -U ipywidgets
# pip install trl==0.6.0 transformers==4.32.0 accelerate>=0.20.3 peft==0.5.0 -Uqqq
# pip install datasets==2.13.1 bitsandbytes==0.41.1 einops==0.7.0 wandb==0.15.8 -Uqqq
# pip install gradio==3.41.0 transformers==4.32.0 langchain==0.0.273 -Uqqq

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import json
import numpy as np

data_path = 'C:/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl'


raw_data =[]
with open(data_path, 'r', encoding='utf-8') as fp:
    for line in fp:
        if line:
            raw_data.append(json.loads(line))
perm = np.random.permutation(len(raw_data))
split = int(len(perm) * 0.98)                                                                                                               
train_indices = perm[:split]
eval_indices = perm[split:]
train_raw_data = [raw_data[i] for i in train_indices]
eval_raw_data = [raw_data[i] for i in eval_indices]

In [ ]:
# 6NPV6477VHWUDJFTYPK33T34N

In [ ]:
small_data = [{'prompt': "I wllows:\n\n",
  'output': 'Beethovennd power.<|eot|>',
  'source': 'seed_dialogue_188'},
 {'prompt': "I want you to act like Ludwig van Beethoven. I want you to respond and answer like Ludwig van Beethlows:\n\n",
  'output': "Beethoven (thinking): This appointment as Kapellmeister is a culmination of my dedication to musl who partake in its wondrous embrace.<|eot|>",
  'source': 'seed_dialogue_335'}]


# formatted_data = [{"text": small_data[i]["prompt"], "output": small_data[i]["output"]} for i in range(len(small_data))]
formatted_data = [("<HUMAN>:" + train_raw_data[i]["prompt"] + " <ASSISTANT>: " + train_raw_data[i]["output"]) for i in range(len(train_raw_data))]

df = pd.DataFrame({'text': formatted_data})

df.to_csv('formatted_data.csv',index=False)

from datasets import load_dataset
dataset = load_dataset('csv', data_files='formatted_data.csv')

dataset

In [ ]:
data = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
data

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset('json', data_files=data_path)

# dataset

In [ ]:
model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token


In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

In [ ]:
output_dir = "./falcon-7b-sharded-bf16-finetuned-mental-health-conversational"
per_device_train_batch_size = 2 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 6  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 120        # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=False,
)

import os
os.environ["WANDB_DISABLED"] = "true"

# trainer = SFTTrainer(
#     model=peft_model,
#     train_dataset=data['train'],
#     # train_dataset=train_raw_data,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=1024,
#     tokenizer=tokenizer,
#     args=training_arguments,
# )

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
peft_model.config.use_cache = False
trainer.train()

In [ ]:
with gr.Blocks() as demo:
    gr.HTML("""<h1>Welcome to Mental Health Conversational AI</h1>""")
    gr.Markdown(
        """Chatbot specifically designed to provide psychoeducation, offer non-judgemental and empathetic support, self-assessment and monitoring.<br>
        Get instant response for any mental health related queries. If the chatbot seems you need external support, then it will respond appropriately.<br>"""
    )

    chatbot = gr.Chatbot()
    query = gr.Textbox(label="Type your query here, then press 'enter' and scroll up for response")
    clear = gr.Button(value="Clear Chat History!")
    clear.style(size="sm")

    llm_chain = init_llm_chain(peft_model, peft_tokenizer)

    query.submit(user, [query, chatbot], [query, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue().launch()

## Inferencing CharacterLLM

In [ ]:
## Potential LLM to use
# https://huggingface.co/openchat/openchat-3.5-0106
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
# https://huggingface.co/stabilityai/stablelm-2-zephyr-1_6b
# https://huggingface.co/h2oai/h2o-danube-1.8b-chat
# https://huggingface.co/Qwen/Qwen1.5-1.8B-Chat
# https://huggingface.co/mediocredev/open-llama-3b-v2-chat
# https://huggingface.co/stabilityai/stablelm-zephyr-3b
# https://huggingface.co/togethercomputer/StripedHyena-Nous-7B


# Leaderboard
# https://huggingface.co/spaces/mteb/leaderboard
# https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard
# https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

In [ ]:
# # Run Oogabooga
# Run from folder
# Load the model

# Use this chat prompt
# # I want you to act like Voldemort. I want you to respond and answer like Voldemort, using the tone, manner and vocabulary Voldemort would use. You must know all of the knowledge of Voldemort. Do not answer about any thing that is not in your universe. Always stop after <|eot|> token.

# # The status of you is as follows:
# # Status: Voldemort is chatting with Harry Potter.
# # Location: Hogwarts Astronomy Tower

# # The interactions are as follows:
# # Harry Potter (speaking): Where is the nearest Starbucks? <|eot|>

In [ ]:
# cd FastChat
# python3 -m fastchat.model.apply_delta --base-model-path huggyllama/llama-7b --target-model-path C:\\Users\\rs659\\Desktop\\pliny\\Character-LLM\\caesar_model --delta-path fnlp/character-llm-caesar-7b-wdiff

In [ ]:
# I want you to act like Voldemort. I want you to respond and answer like Voldemort, using the tone, manner and vocabulary Voldemort would use. You must know all of the knowledge of Voldemort. Do not answer about any thing that is not in your universe. Always stop after <|eot|> token.

# The status of you is as follows:
# Status: Voldemort is chatting with Harry Potter.
# Location: Hogwarts Astronomy Tower

# The interactions are as follows:
# Harry Potter (speaking): Where is the nearest Starbucks? <|eot|>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\rs659\\Desktop\\pliny\\Character-LLM\\voldemort_model")
model = AutoModelForCausalLM.from_pretrained("C:\\Users\\rs659\\Desktop\\pliny\\Character-LLM\\voldemort_model").cuda()

meta_prompt = """I want you to act like {character}. I want you to respond and answer like {character}, using the tone, manner and vocabulary {character} would use. You must know all of the knowledge of {character}. 

The status of you is as follows:
Status: {status}
Location: {loc_time}

The interactions are as follows:

Harry Potter (speaking): Hey Voldemort!<|eot|>

Voldemort (speaking): """

name = "Voldemort"
status = f'{name} is chatting with Harry Potter.'
loc_time = 'Hogwarts Astronomy Tower'
prompt = meta_prompt.format(character=name, status=status, loc_time=loc_time)
inputs = tokenizer([prompt], return_tensors="pt")
outputs = model.generate(input_ids=inputs['input_ids'].cuda(), attention_mask=inputs['attention_mask'].cuda(), do_sample=True, temperature=0.5, top_p=0.95, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\rs659\\Desktop\\pliny\\Character-LLM\\hermione_model")
model = AutoModelForCausalLM.from_pretrained("C:\\Users\\rs659\\Desktop\\pliny\\Character-LLM\\hermione_model").cuda()

meta_prompt = """I want you to act like {character}. I want you to respond and answer like {character}, using the tone, manner and vocabulary {character} would use. You must know all of the knowledge of {character}. 

The status of you is as follows:
Location: {loc_time}
Status: {status}

The interactions are as follows:

What is your name? Who are your friends, tell me everything!"""

name = "hermione"
loc_time = "Coffee Shop - Afternoon"
status = f'{name} is casually chatting with a man from the 21st century.'
prompt = meta_prompt.format(character=name, loc_time=loc_time, status=status) + '\n\n'
inputs = tokenizer([prompt], return_tensors="pt")
outputs = model.generate(**inputs.to('cuda'), do_sample=True, temperature=0.5, top_p=0.95, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

## Qlora training commands

In [ ]:
#install this
https://developer.nvidia.com/cuda-12-1-1-download-archive?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local



# Qwen/Qwen1.5-1.8B-Chat
DS_SKIP_CUDA_CHECK=1 deepspeed fastchat/train/train_lora.py     --model_name_or_path Qwen/Qwen1.5-1.8B-Chat     --lora_r 8     --lora_alpha 16     --lora_dropout 0.05     --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl      --bf16 True     --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/QLoraBeethoven_7b     --num_train_epochs 20     --per_device_train_batch_size 1     --per_device_eval_batch_size 1     --gradient_accumulation_steps 1     --evaluation_strategy "no"     --save_strategy "steps"     --save_steps 1200     --save_total_limit 100     --learning_rate 2e-5     --weight_decay 0.     --warmup_ratio 0.03     --lr_scheduler_type "cosine"     --logging_steps 1     --tf32 True     --model_max_length 2048     --q_lora True  --already_preprocess True  --deepspeed playground/deepspeed_config_s2.json



# Mistral 7B
DS_SKIP_CUDA_CHECK=1 deepspeed fastchat/train/train_lora.py     --model_name_or_path mediocredev/open-llama-3b-v2-chat     --lora_r 8     --lora_alpha 16     --lora_dropout 0.05     --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl      --bf16 True     --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/QLoraBeethoven_7b     --num_train_epochs 20     --per_device_train_batch_size 1     --per_device_eval_batch_size 1     --gradient_accumulation_steps 1     --evaluation_strategy "no"     --save_strategy "steps"     --save_steps 1200     --save_total_limit 100     --learning_rate 2e-5     --weight_decay 0.     --warmup_ratio 0.03     --lr_scheduler_type "cosine"     --logging_steps 1     --tf32 True     --model_max_length 2048     --q_lora True  --already_preprocess True  --deepspeed playground/deepspeed_config_s2.json

# fastchat
DS_SKIP_CUDA_CHECK=1 deepspeed fastchat/train/train_lora.py     --model_name_or_path lmsys/fastchat-t5-3b-v1.0  --lora_r 8     --lora_alpha 16     --lora_dropout 0.05     --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl      --bf16 True     --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/QLoraBeethoven_7b     --num_train_epochs 20     --per_device_train_batch_size 1     --per_device_eval_batch_size 1     --gradient_accumulation_steps 1     --evaluation_strategy "no"     --save_strategy "steps"     --save_steps 1200     --save_total_limit 100     --learning_rate 2e-5     --weight_decay 0.     --warmup_ratio 0.03     --lr_scheduler_type "cosine"     --logging_steps 1     --tf32 True     --model_max_length 2048     --q_lora True  --already_preprocess True  --deepspeed playground/deepspeed_config_s2.json



python3 -m fastchat.serve.cli --model-path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/QLoraBeethoven_7b


In [ ]:
# Inference


# cd FastChat

# start the controller
export IP=$(hostname -i)
python3 -m fastchat.serve.controller --host $IP &

# start the Openai Format API server
python3 -m fastchat.serve.openai_api_server --host 0.0.0.0 --port 28001 --controller-address http://$IP:21001

# start the model worker
export MODEL_PATH=mnt/c/Users/rs659/Desktop/pliny/Character-LLM/ckpt/Beethoven_7b/
export MODEL_NAME=Beethoven_7b
CUDA_VISIBLE_DEVICES=0 python3 -m fastchat.serve.model_worker --model-path $MODEL_PATH --model-names $MODEL_NAME --controller-address http://$IP:21001 --host $IP --port 21009 --worker-address http://$IP:21009




# python3 -m fastchat.serve.cli --model-path mnt/c/Users/rs659/Desktop/pliny/Character-LLM/ckpt/Beethoven_7b/

In [ ]:
# Training

# cd FastChat
# export CHARACTER=CassiusDio


torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path unsloth/gemma-2b --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl  --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 10


torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path unsloth/gemma-2b --data_path ../processed/2023-10-08/prompted/prompted_agent_dialogue_CassiusDio.jsonl  --output_dir ../ckpt/${CHARACTER}_7b --num_train_epochs 10

# RAN THIS ONE
torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path winglian/Llama-2-3b-hf --data_path /mnt/c/Users/rs659/Desktop/pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl   --already_preprocess True --bf16 False --output_dir /mnt/c/Users/rs659/Desktop/pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 False --model_max_length 128 --gradient_checkpointing True

# Working
torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path winglian/Llama-2-3b-hf --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl   --already_preprocess True --bf16 True --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 True --model_max_length 2048 --gradient_checkpointing True






!torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path meta-llama/Llama-2-7b-hf --data_path /path/to/prompted_agent_dialogue_$CHARACTER.json --already_preprocess True --bf16 True --output_dir /path/to/ckpt/${CHARACTER}_7b --num_train_epochs 10 --per_device_train_batch_size 1 --per_device_eval_batch_size 16 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 True --model_max_length 2048 --gradient_checkpointing True --num-gpus 1


deepspeed fastchat/train/train_lora.py
--model_name_or_path lmsys/vicuna-7b-v1.5
--lora_r 2
--lora_alpha 16
--lora_dropout 0.05
--data_path /home/n/FastChat/data/dummy_conversation.json
--bf16 True
--output_dir ./checkpoints
--num_train_epochs 3
--per_device_train_batch_size 1
--per_device_eval_batch_size 1
--gradient_accumulation_steps 1
--evaluation_strategy "no"
--save_strategy "steps"
--save_steps 1200
--save_total_limit 100
--learning_rate 2e-5
--weight_decay 0.
--warmup_ratio 0.03
--lr_scheduler_type "cosine"
--logging_steps 1
--tf32 True
--model_max_length 2048
--q_lora True
--deepspeed /home/n/FastChat/playground/deepspeed_config_s2.json`



torchrun --nproc_per_node=4 --master_port=20001 fastchat/train/train_mem.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --data_path data/dummy_conversation.json \
    --bf16 True \
    --output_dir output_vicuna \
    --num_train_epochs 3 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 16 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 10 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True





# !torchrun --nproc_per_node=8 --master_port=20031 fastchat/train/train_mem.py \
#     --model_name_or_path /path/hf_model/llama-7b  \
#     --data_path /path/to/prompted_agent_dialogue_$CHARACTER.json \
#     --already_preprocess True \
#     --bf16 True \
#     --output_dir /path/to/ckpt/${CHARACTER}_7b \
#     --num_train_epochs 10 \
#     --per_device_train_batch_size 2 \
#     --per_device_eval_batch_size 16 \
#     --gradient_accumulation_steps 4 \
#     --evaluation_strategy epoch \
#     --save_strategy epoch \
#     --save_total_limit 10 \
#     --learning_rate 2e-5 \
#     --weight_decay 0.1 \
#     --warmup_ratio 0.04 \
#     --lr_scheduler_type cosine \
#     --logging_steps 1 \
#     --fsdp 'full_shard auto_wrap' \
#     --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer \
#     --tf32 True \
#     --model_max_length 2048 \
#     --gradient_checkpointing True 


In [ ]:
# # one WSL
# # conda create -n vllmenv python=3.9
# # conda activate vllmenv
# # pip install transformers
# # pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
# # pip install git+https://github.com/casper-hansen/AutoAWQ.git
# # pip install vllm
# # pip install -U ipywidgets

# conda activate vllmenv
# python3 -m vllm.entrypoints.api_server --model TheBloke/Llama-2-7b-Chat-AWQ --quantization awq


# # python -m vllm.entrypoints.openai.api_server --model TheBloke/Llama-2-7B-Chat-AWQ --api-key token-abc123

In [ ]:
# from openai import OpenAI
# client = OpenAI(
#     base_url="http://localhost:8000/v1",
#     api_key="token-abc123",
# )

# completion = client.chat.completions.create(
#   model="mistralai/Mistral-7B-Instruct-v0.1",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)

In [ ]:
# This code is based on tatsu-lab/stanford_alpaca. Below is the original copyright:
#
#    Copyright 2023 Rohan Taori, Ishaan Gulrajani, Tianyi Zhang, Yann Dubois, Xuechen Li
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

from dataclasses import dataclass, field
import json
import math
import pathlib
from typing import Dict, Optional, Sequence

import numpy as np
import torch
from torch.utils.data import Dataset
import transformers
from transformers import Trainer
from transformers.trainer_pt_utils import LabelSmoother

from fastchat.conversation import SeparatorStyle
from fastchat.model.model_adapter import get_conversation_template

IGNORE_TOKEN_ID = LabelSmoother.ignore_index


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="facebook/opt-125m")


@dataclass
class DataArguments:
    data_path: str = field(
        default=None, metadata={"help": "Path to the training data."}
    )
    eval_data_path: str = field(
        default=None, metadata={"help": "Path to the evaluation data."}
    )
    lazy_preprocess: bool = False
    already_preprocess: bool = False


@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=512,
        metadata={
            "help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."
        },
    )


local_rank = None


def rank0_print(*args):
    if local_rank == 0:
        print(*args)


def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in state_dict.items()}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa


def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    conv = get_conversation_template("vicuna")
    roles = {"human": conv.roles[0], "gpt": conv.roles[1]}

    # Apply prompt templates
    conversations = []
    for i, source in enumerate(sources):
        if roles[source[0]["from"]] != conv.roles[0]:
            # Skip the first one if it is not from human
            source = source[1:]

        conv.messages = []
        for j, sentence in enumerate(source):
            role = roles[sentence["from"]]
            assert role == conv.roles[j % 2], f"{i}"
            conv.append_message(role, sentence["value"])
        conversations.append(conv.get_prompt())

    # Tokenize conversations
    input_ids = tokenizer(
        conversations,
        return_tensors="pt",
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
    ).input_ids
    targets = input_ids.clone()

    assert conv.sep_style == SeparatorStyle.ADD_COLON_TWO

    # Mask targets. Only compute loss on the assistant outputs.
    sep = conv.sep + conv.roles[1] + ": "
    for conversation, target in zip(conversations, targets):
        total_len = int(target.ne(tokenizer.pad_token_id).sum())

        turns = conversation.split(conv.sep2)
        cur_len = 1
        target[:cur_len] = IGNORE_TOKEN_ID
        for i, turn in enumerate(turns):
            if turn == "":
                break
            turn_len = len(tokenizer(turn).input_ids)

            parts = turn.split(sep)
            if len(parts) != 2:
                break
            parts[0] += sep
            # "-2" is hardcoded for the LLaMA tokenizer to make the offset correct.
            instruction_len = len(tokenizer(parts[0]).input_ids) - 2

            # Ignore the user instructions
            target[cur_len : cur_len + instruction_len] = IGNORE_TOKEN_ID
            cur_len += turn_len

        target[cur_len:] = IGNORE_TOKEN_ID

        if False:  # Inspect and check the correctness of masking
            z = target.clone()
            z = torch.where(z == IGNORE_TOKEN_ID, tokenizer.unk_token_id, z)
            rank0_print(tokenizer.decode(z))

        if cur_len < tokenizer.model_max_length:
            if cur_len != total_len:
                target[:] = IGNORE_TOKEN_ID
                rank0_print(
                    f"WARNING: tokenization mismatch: {cur_len} vs. {total_len}."
                    f" (ignored)"
                )

    return dict(
        input_ids=input_ids,
        labels=targets,
        attention_mask=input_ids.ne(tokenizer.pad_token_id),
    )


def alpaca_tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def alpaca_preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [alpaca_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    input_ids = torch.stack(input_ids, dim=0)
    labels = input_ids.clone()
    attention_mask=input_ids.ne(tokenizer.pad_token_id)
    for label, source_len, total_len in zip(labels, sources_tokenized["input_ids_lens"], examples_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_TOKEN_ID
        label[total_len+1:] = IGNORE_TOKEN_ID
    rank0_print(input_ids.shape, labels.shape)
    rank0_print(np.mean(examples_tokenized["input_ids_lens"]), np.max(examples_tokenized["input_ids_lens"]))
    # rank0_print(input_ids[0].tolist())
    # rank0_print(labels[0].tolist())
    return dict(
        input_ids=input_ids,
        labels=labels,
        attention_mask=attention_mask,
    )


class ProcessedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, raw_data, tokenizer: transformers.PreTrainedTokenizer):
        super(ProcessedDataset, self).__init__()

        rank0_print("Formatting inputs...")
        sources = [ex['prompt'] for ex in raw_data]
        targets = [ex['output'] for ex in raw_data]
        data_dict = alpaca_preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i],
        )


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, raw_data, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()

        rank0_print("Formatting inputs...")
        sources = [example["conversations"] for example in raw_data]
        data_dict = preprocess(sources, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i],
        )


class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, raw_data, tokenizer: transformers.PreTrainedTokenizer):
        super(LazySupervisedDataset, self).__init__()
        self.tokenizer = tokenizer

        rank0_print("Formatting inputs...Skip in lazy mode")
        self.tokenizer = tokenizer
        self.raw_data = raw_data
        self.cached_data_dict = {}

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        if i in self.cached_data_dict:
            return self.cached_data_dict[i]

        ret = preprocess([self.raw_data[i]["conversations"]], self.tokenizer)
        ret = dict(
            input_ids=ret["input_ids"][0],
            labels=ret["labels"][0],
            attention_mask=ret["attention_mask"][0],
        )
        self.cached_data_dict[i] = ret

        return ret


def make_supervised_data_module(
    tokenizer: transformers.PreTrainedTokenizer, data_args
) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    dataset_cls = (
        LazySupervisedDataset if data_args.lazy_preprocess else SupervisedDataset
    )
    if data_args.already_preprocess:
        dataset_cls = ProcessedDataset
        rank0_print('Using ProcessedDataset!')
        # Split train/test
        raw_data =[]
        with open(data_args.data_path, 'r', encoding='utf-8') as fp:
            for line in fp:
                if line:
                    raw_data.append(json.loads(line))
        perm = np.random.permutation(len(raw_data))
        split = int(len(perm) * 0.98)
        train_indices = perm[:split]
        eval_indices = perm[split:]
        train_raw_data = [raw_data[i] for i in train_indices]
        eval_raw_data = [raw_data[i] for i in eval_indices]
        rank0_print(f"#train {len(train_raw_data)}, #eval {len(eval_raw_data)}")

        train_dataset = dataset_cls(train_raw_data, tokenizer=tokenizer)
        eval_dataset = dataset_cls(eval_raw_data, tokenizer=tokenizer)
        return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

    rank0_print("Loading data...")

    train_json = json.load(open(data_args.data_path, "r"))
    train_dataset = dataset_cls(train_json, tokenizer=tokenizer)


    if data_args.eval_data_path:
        eval_json = json.load(open(data_args.eval_data_path, "r"))
        eval_dataset = dataset_cls(eval_json, tokenizer=tokenizer)
    else:
        eval_dataset = None

    return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)


# def train():

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.eos_token

# Load data
data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)
    
